A1.

---



In [ ]:
pip install requests beautifulsoup4 msticpy

In [2]:
import requests
from bs4 import BeautifulSoup
import re

def iocs(url):
    try:
        # HTTP request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract URLs
        pattern = re.compile(r'(?<=href=\")(?!#)(.+?)(?=#|\")')
        urls = re.findall(pattern, response.text)

        # Extract IP addresses
        ip_addresses = re.findall(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', response.text)

        # Extract MD5 hashes of content
        md5_hashes = re.findall(r'[a-fA-F0-9]{32}', response.text)

        # Extract SHA-1 hashes
        sha1_hashes = re.findall(r'\b[a-fA-F0-9]{40}\b', response.text)

        # Extract SHA-256 hashes
        sha256_hashes = re.findall(r'\b[a-fA-F0-9]{64}\b', response.text)

        # Extracted IOCs
        print("Extracted URLs:", urls)
        print("Extracted IP Addresses:", ip_addresses)
        print("Extracted MD5 Hashes:", md5_hashes)
        print("Extracted SHA1 Hashes:", sha1_hashes)
        print("Extracted SHA256 Hashes:", sha256_hashes)

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")


url = 'https://www.secureworks.com/blog/opsec-mistakes-reveal-cobalt-mirage-threat-actors'
iocs(url)

Extracted URLs: ['https://www.secureworks.com/blog/opsec-mistakes-reveal-cobalt-mirage-threat-actors', '//ajax.googleapis.com', '//apis.google.com', '//www.google-analytics.com', '//player.vimeo.com', '//munchkin.marketo.net', 'https://prod-www.secureworks.com/favicon/site.webmanifest', 'https://prod-www.secureworks.com/favicon/apple-touch-icon.png', 'https://prod-www.secureworks.com/favicon/favicon-32x32.png', 'https://prod-www.secureworks.com/favicon/favicon-16x16.png', 'https://prod-www.secureworks.com/favicon/safari-pinned-tab.svg', 'https://prod-www.secureworks.com/favicon/favicon.ico', 'https://fonts.googleapis.com', 'https://fonts.gstatic.com/', 'https://fonts.gstatic.com', 'https://fonts.googleapis.com/css2?family=Figtree:ital,wght@0,400;0,600;1,300&family=Saira+Condensed:wght@300;400;600&display=swap', '/resources/rp-state-of-the-threat-2023', '/contact/emergency-response', '/customer-support', '/blog', '/', '/contact/request-demo-xdr', 'https://www.secureworks.com/research/th

A2.

---


In [ ]:
pip install python-whois

In [4]:
import requests
from bs4 import BeautifulSoup
import whois
import csv

def extract_domains_from_webpage(url):
    try:
        # HTTP request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')

        # Extract unique domains
        domains = set()
        for link in links:
            href = link.get('href')
            if href:
                domain = extract_domain_from_url(href)
                if domain:
                    domains.add(domain)

        return list(domains)

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []

def extract_domain_from_url(url):
    return url.split("//")[-1].split("/")[0].split("?")[0].split("#")[0].split(":")[0]

def get_whois_information(domains):
    whois_info_list = []
    for domain in domains:
        try:
            whois_info = whois.whois(domain)
            whois_info_list.append({
                'Domain': domain,
                'Domain Name': whois_info.domain_name,
                'Registrar': whois_info.registrar,
                'WHOIS Server': whois_info.whois_server,
                'Referral URL': whois_info.referral_url,
                'Update Date': whois_info.update_date,
                'Creation Date': whois_info.creation_date,
                'Expiration Date': whois_info.expiration_date,
                'Name Servers': whois_info.name_servers,
                'Status': whois_info.status,
                'Emails': whois_info.emails,
                'DNSSEC': whois_info.dnssec,
                'Name': whois_info.name,
                'Org': whois_info.org,
                'Address': whois_info.address,
                'City': whois_info.city,
                'State': whois_info.state,
                'Registrant Postal Code': whois_info.registrant_postal_code,
                'Country': whois_info.country,
            })
        except whois.parser.PywhoisError as e:
            print(f"Error retrieving WHOIS information for {domain}: {e}")

    return whois_info_list

def write_to_csv(data, output_file):
    fieldnames = ['Domain', 'Domain Name', 'Registrar', 'WHOIS Server', 'Referral URL', 'Update Date', 'Creation Date', 'Expiration Date',
                  'Name Servers', 'Status', 'Emails', 'DNSSEC', 'Name', 'Org', 'Address', 'City', 'State', 'Registrant Postal Code',
                  'Country']

    with open(output_file, mode='w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

webpage_url = 'https://www.secureworks.com/blog/opsec-mistakes-reveal-cobalt-mirage-threat-actors'
output_file = 'whois_information.csv'

# Extract domains from the webpage
extracted_domains = extract_domains_from_webpage(webpage_url)
print(extracted_domains)

# Get WHOIS information for each domain
whois_info_list = get_whois_information(extracted_domains)

# Write to a CSV file
write_to_csv(whois_info_list, output_file)


['www.justice.gov', 'home.treasury.gov', 'nvd.nist.gov', 'www.mandiant.com', 'portal.secureworks.com', 'www.washingtoninstitute.org', 'github.com', 'www.secureworks.com', 'www.microsoft.com', 'www.zerodayinitiative.com', 'investors.secureworks.com', 'javascript', 'www.linkedin.com', 'pages.secureworks.com', 'twitter.com', 'www.facebook.com']


In [5]:
import pandas as pd
df = pd.read_csv('whois_information.csv')
df.head()

,Domain,Domain Name,Registrar,WHOIS Server,Referral URL,Update Date,Creation Date,Expiration Date,Name Servers,Status,Emails,DNSSEC,Name,Org,Address,City,State,Registrant Postal Code,Country
0,www.justice.gov,justice.gov,Cybersecurity and Infrastructure Security Agency,whois.cloudflareregistry.com,NaN,NaN,1998-05-14 14:24:37,2024-09-07 13:04:21,"['ns-jdcw-01.usdoj.gov', 'ns-jdcw-02.usdoj.gov']",serverTransferProhibited https://icann.org/epp...,NaN,signedDelegation,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY
1,home.treasury.gov,treasury.gov,Cybersecurity and Infrastructure Security Agency,whois.cloudflareregistry.com,NaN,NaN,1998-03-07 01:38:18,2024-10-07 14:03:31,"['ns01.treasury.gov', 'ns02.treasury.gov']",serverTransferProhibited https://icann.org/epp...,NaN,unsigned,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY
2,nvd.nist.gov,nist.gov,Cybersecurity and Infrastructure Security Agency,whois.cloudflareregistry.com,NaN,NaN,1997-10-02 01:29:27,2024-09-23 14:27:56,"['a1-216.akam.net', 'a12-65.akam.net', 'a13-67...",serverTransferProhibited https://icann.org/epp...,NaN,signedDelegation,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY
3,www.mandiant.com,"['MANDIANT.COM', 'mandiant.com']","MarkMonitor, Inc.",whois.markmonitor.com,NaN,NaN,2005-12-22 21:13:06,2026-12-22 21:13:06,"['JADE.NS.CLOUDFLARE.COM', 'JOEL.NS.CLOUDFLARE...",['clientDeleteProhibited https://icann.org/epp...,"['abusecomplaints@markmonitor.com', 'whoisrequ...",signedDelegation,NaN,Google LLC,NaN,NaN,CA,NaN,US
4,portal.secureworks.com,SECUREWORKS.COM,"GoDaddy.com, LLC",whois.godaddy.com,NaN,NaN,"[datetime.datetime(1997, 6, 9, 4, 0), datetime...","[datetime.datetime(2025, 6, 18, 11, 59, 59), d...","['EDNS2.ULTRADNS.BIZ', 'EDNS2.ULTRADNS.COM', '...",['clientDeleteProhibited https://icann.org/epp...,abuse@godaddy.com,unsigned,Registration Private,"Domains By Proxy, LLC","['DomainsByProxy.com', '2155 E Warner Rd']",Tempe,Arizona,85284,US


B1.

1. WHOIS Information extracted from IP Addresses and Domains can provides details about domain ownership, registration dates, and contact information, aiding in the identification of malicious actors and infrastructure.

2. Geolocation Data can be obtain from IP Addresses which can help us understand the potential origin of cyber threats.

3. VirusTotal API can be used to provide valuable insights into the reputation and potential maliciousness of IOCs, able to analyze URLs, IP Addresses, hashes and domains.

4. IP Reputation Analysis can be used to determine its trustworthiness. Every IP Address is assigned a reputation score based on historical data.

B2.

1. Anomaly Detection. Utilize anomaly detection techniques to identify IOCs that deviate from the normal patterns observed in your network or environment. This may include unusual network traffic, unexpected communication patterns, or uncommon behaviors.

2. Train machine learning models to identify unknown IOCs based on historical data. Machine learning algorithms can learn patterns and anomalies that may not be apparent through traditional methods.

3. Use sandbox environments to analyze files or URLs associated with the IOCs. Sandboxing allows us to observe the behavior of potentially malicious code in a controlled environment and identify new threats.

C.

---



In [6]:
import requests

# Download file
url = "https://www.secrepo.com/maccdc2012/http.log.gz"
filename = url.split("/")[-1]
with open(filename, "wb") as f:
    r = requests.get(url)
    f.write(r.content)

In [7]:
import shutil
import gzip

# Unzip File
file_name = 'http.log.gz'
with gzip.open(file_name,'rb') as f_in:
    with open(file_name[:-3],'wb') as f_out:
        shutil.copyfileobj(f_in,f_out)

In [8]:
# Look at first 10 lines
f = open('http.log', 'r')
lines = f.readlines()
for i in range(10):
    print(lines[0])

1331901000.000000	CHEt7z3AzG4gyCNgci	192.168.202.79	50465	192.168.229.251	80	1	HEAD	192.168.229.251	/DEASLog02.nsf	-	Mozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.org/book/nse.html)	0	0	404	Not Found	-	-	-	(empty)	-	-	-	-	-	-	-

1331901000.000000	CHEt7z3AzG4gyCNgci	192.168.202.79	50465	192.168.229.251	80	1	HEAD	192.168.229.251	/DEASLog02.nsf	-	Mozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.org/book/nse.html)	0	0	404	Not Found	-	-	-	(empty)	-	-	-	-	-	-	-

1331901000.000000	CHEt7z3AzG4gyCNgci	192.168.202.79	50465	192.168.229.251	80	1	HEAD	192.168.229.251	/DEASLog02.nsf	-	Mozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.org/book/nse.html)	0	0	404	Not Found	-	-	-	(empty)	-	-	-	-	-	-	-

1331901000.000000	CHEt7z3AzG4gyCNgci	192.168.202.79	50465	192.168.229.251	80	1	HEAD	192.168.229.251	/DEASLog02.nsf	-	Mozilla/5.0 (compatible; Nmap Scripting Engine; http://nmap.org/book/nse.html)	0	0	404	Not Found	-	-	-	(empty)	-	-	-	-	-	-	-

1331901000.000000	CHEt7z3AzG

In [9]:
import pandas as pd

# Assumption ip is the sender, ip2 is the web server
data = pd.read_csv(
    'http.log',
    sep=r"\t",
    engine='python',
    header=None,
    usecols=[2, 4],
    names=['ip', 'ip2'],)
data.head()

,ip,ip2
0,192.168.202.79,192.168.229.251
1,192.168.202.79,192.168.229.251
2,192.168.202.79,192.168.229.251
3,192.168.202.79,192.168.229.251
4,192.168.202.79,192.168.229.251


In [14]:
# Assumption reconnaissance activities means sending request more than 1000 times regardless of time of request
data1 = data.value_counts().loc[lambda x: x>1000].index
data1

MultiIndex([( '192.168.203.63', '192.168.229.101'),
            ( '192.168.202.79',  '192.168.25.203'),
            ('192.168.202.102',  '192.168.23.202'),
            ('192.168.202.102',  '192.168.26.202'),
            ('192.168.202.102',  '192.168.24.202'),
            ('192.168.202.110',  '192.168.27.203'),
            ('192.168.202.102',  '192.168.24.101'),
            ('192.168.202.102',  '192.168.28.202'),
            ('192.168.202.110', '192.168.229.251'),
            ('192.168.202.110',  '192.168.22.253'),
            ('192.168.202.110',  '192.168.27.253'),
            ('192.168.202.110',  '192.168.22.102'),
            ('192.168.202.110',  '192.168.28.202'),
            ('192.168.202.138',  '192.168.21.102'),
            ('192.168.202.138',  '192.168.27.102'),
            ('192.168.202.110',  '192.168.27.102'),
            ('192.168.202.110', '192.168.229.156'),
            ('192.168.202.102',  '192.168.22.202'),
            ( '192.168.202.79', '192.168.229.101'),
            

In [12]:
possible_IPs = []
for i in data1:
    possible_IPs.append(i[0])
print(possible_IPs)

['192.168.203.63', '192.168.202.79', '192.168.202.102', '192.168.202.102', '192.168.202.102', '192.168.202.110', '192.168.202.102', '192.168.202.102', '192.168.202.110', '192.168.202.110', '192.168.202.110', '192.168.202.110', '192.168.202.110', '192.168.202.138', '192.168.202.138', '192.168.202.110', '192.168.202.110', '192.168.202.102', '192.168.202.79', '192.168.202.110', '192.168.202.110', '192.168.202.138', '192.168.202.138', '192.168.202.125', '192.168.202.102', '192.168.202.65', '192.168.202.102', '192.168.202.138', '192.168.202.110', '192.168.202.118', '192.168.202.110', '192.168.202.110', '192.168.202.110', '192.168.202.110', '192.168.202.138', '192.168.202.76', '192.168.202.118', '192.168.202.118', '192.168.202.96', '192.168.202.118', '192.168.202.79', '192.168.202.150', '192.168.202.125', '192.168.202.138', '192.168.202.110', '192.168.202.140', '192.168.202.140', '192.168.202.140', '192.168.202.140', '192.168.202.110', '192.168.202.96', '192.168.202.110', '192.168.202.110', 

In [17]:
IPs = list(set(possible_IPs))
print(IPs)
print(len(IPs))

['192.168.203.63', '192.168.202.76', '192.168.202.87', '192.168.202.65', '192.168.204.70', '192.168.202.79', '192.168.202.96', '192.168.202.125', '192.168.202.150', '192.168.202.102', '192.168.202.140', '192.168.202.90', '192.168.202.118', '192.168.202.112', '192.168.202.110', '192.168.202.138']
16
